In [2]:
import pandas as pd
import numpy as np
import json
df = json.load(open("clusts/cs_23_I_max_2hp_10gc_new2_mfe_5000.json"))

In [3]:
out = {}
print(df)
for key, value in df.items():
    print(f"{key}, min {np.mean(value)}, std {np.std(value)}, max {np.max(value)}")
    out[key] = np.mean(value)
#print(df)

In [4]:
for key, value in df.items():
    # create a violin plot usin seaborn:
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    sns.violinplot(data=df[key], ax=ax)
    print(key)
    try:
        conf_name = key.split('sleeping_beauty_150_')[1] 
        key_title = conf_name.replace("compress_encrypt_high_entropy_dist","high entropy,").replace("bmp_low_entropy_diff_dist", "diff low entropy,").replace("bmp_low_entropy_evo_dist", "evo low entropy,").replace("raptor", "default (raptor),")
    except:
        conf_name = key.replace("clusts/", "")
        key_title = conf_name
    key_title = key_title.replace("sleeping_beauty_", "").replace("seedspacing", "spacing ").replace('_', ' ').replace(" payloadxor", ", payloadxor").replace('.fasta', '').replace("grass", "grass code").replace("cs23", "")
        # Add the mean value to the plot using plt.text()
    plt.text(0.8, 0.9, f"Mean: {out[key]:.5f}", transform=ax.transAxes, fontsize=12, ha='center')
    
    #plt.title(f"{key_title}")
    plt.title("Minimum free energy (MFE) of the generated sequences")
    plt.xlabel(key_title, fontsize=12)
    plt.grid(True)
    plt.savefig(f"clusts/mfe_plot/mfe_violin_{conf_name}.svg", format="svg", dpi=1200)
    plt.savefig(f"clusts/mfe_plot/mfe_violin_{conf_name}.pdf", bbox_inches="tight")
    plt.show()
    

In [52]:
fig, ax = plt.subplots()
sns.violinplot(data=df, ax=ax)
plt.show()
plt.close()
data = []
for key, value in df.items():
    # create a violin plot usin seaborn:
    value = sorted(value, reverse=True) #[:3000]
    import seaborn as sns
    import matplotlib.pyplot as plt
    #fig, ax = plt.subplots()
    #sns.violinplot(data=value, ax=ax)
    print(key)
    try:
        conf_name = key.split('sleeping_beauty_150_')[1] 
        key_title = conf_name.replace("compress_encrypt_high_entropy_dist","high entropy,").replace("bmp_low_entropy_diff_dist", "diff low entropy,").replace("bmp_low_entropy_evo_dist", "evo low entropy,").replace("raptor", "default (raptor),")
    except:
        conf_name = key.replace("clusts/", "")
        key_title = conf_name
    key_title = key_title.replace("sleeping_beauty_", "").replace("seedspacing", "spacing ").replace('_', ' ').replace(" payloadxor", ", payloadxor").replace('.fasta', '').replace("grass", "grass code").replace("cs23", "")
    splits = key_title.split(",")
    if len(splits) == 2:
        dist, spacing = splits
        spacing = int(spacing.split("spacing ")[1])
        payloadxor = False
    elif len(splits) == 3:
        dist, spacing, _ = splits
        payloadxor = True
        spacing = int(spacing.split("spacing ")[1])
    else:
        dist = splits[0]
        payloadxor = False
        spacing = 0
    data.append({
        "dist": dist,
        "spacing": spacing,
        "payloadxor": payloadxor,
        "combination": dist + ', spacing ' + str(spacing) + (', payloadxor' if payloadxor else ''),
        "mfe": value
    })
data2 = pd.DataFrame(data)
data2

In [49]:
data2['combination'] = data2['dist'].astype(str) + ', spacing ' + data2['spacing'].astype(str) + (', ' + data2['payloadxor'].astype(str) if data2['payloadxor'].any() else '')
filtered_data = data2[data2['dist'].str.strip() != "dna fountain"]
filtered_data = filtered_data[filtered_data['spacing'] < 5]
filtered_data = filtered_data[filtered_data['payloadxor'] == False]

filtered_raw_data = {x["combination"]: x["mfe"] for x in data if  not "24" in x['dist'] and x['spacing'] <3 and x['payloadxor'] == False}
# plot the data:
plot_order = sorted(filtered_raw_data.keys())
plot_order.remove("dna fountain , spacing 0")
plot_order.append("dna fountain , spacing 0")
fig, ax = plt.subplots()
sns.violinplot(filtered_raw_data, ax=ax, order=plot_order)
plt.title("Comparison: Minimum free energy (MFE) for the different experiments")
ax.xaxis.label.set_rotation(45)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylabel("Minimum Free Energy (MFE)")
ax.grid(True)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_no_payload_xor.svg", format="svg", dpi=1200)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_no_payload_xor.pdf", bbox_inches="tight")
plt.show()
filtered_data

In [53]:
data2['combination'] = data2['dist'].astype(str) + ', spacing ' + data2['spacing'].astype(str) + (', ' + data2['payloadxor'].astype(str) if data2['payloadxor'].any() else '')
filtered_data = data2[data2['dist'].str.strip() != "dna fountain"]
filtered_data = filtered_data[filtered_data['spacing'] < 5]
filtered_data = filtered_data[filtered_data['payloadxor'] == False]

filtered_raw_data = {x["combination"]: x["mfe"] for x in data if  not "24" in x['dist'] and x['spacing'] <3 and x['payloadxor'] == True}
# plot the data:
plot_order = sorted(filtered_raw_data.keys())
#plot_order.remove("dna fountain , spacing 0")
#plot_order.append("dna fountain , spacing 0")
fig, ax = plt.subplots()
sns.violinplot(filtered_raw_data, ax=ax, order=plot_order)
ax.xaxis.label.set_rotation(45)
plt.title("Comparison: Minimum free energy (MFE) with payloadxor")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylabel("Minimum Free Energy (MFE)")
ax.grid(True)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_payload_xor.svg", format="svg", dpi=1200)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_payload_xor.pdf", bbox_inches="tight")
plt.show()
filtered_data

In [41]:
data2['combination'] = data2['dist'].astype(str) + ', spacing ' + data2['spacing'].astype(str) + (', ' + data2['payloadxor'].astype(str) if data2['payloadxor'].any() else '')
filtered_data = data2[data2['dist'].str.strip() != "dna fountain"]
filtered_data = filtered_data[filtered_data['spacing'] < 3]
filtered_data = filtered_data[filtered_data['payloadxor'] == True]

filtered_raw_data = {x["combination"]: sorted(x["mfe"], reverse=True)[:713] for x in data if  not "24" in x['dist'] and x['spacing'] < 3 and (x['payloadxor'] == True or "dna fountain" in x['dist'] or "grass" in x['dist'])}
# plot the data:
plot_order = sorted(filtered_raw_data.keys())
plot_order.remove("dna fountain , spacing 0")
plot_order.append("dna fountain , spacing 0")
fig, ax = plt.subplots()
sns.violinplot(filtered_raw_data, ax=ax, order=plot_order)
ax.xaxis.label.set_rotation(45)
plt.title("Comparison: Minimum free energy (MFE) - best 713 sequences") # best 713 out of the 50.000 generated sequences - it has to be expceted that when generating all 2^32 sequences, this average will improve even further.
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylabel("Minimum Free Energy (MFE)")
ax.grid(True)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_payload_xor_first_187_only.svg", format="svg", dpi=1200)
plt.savefig(f"clusts/mfe_plot/mfe_violin_cmp_payload_xor_first_187_only.pdf", bbox_inches="tight")
plt.show()
filtered_data

In [51]:
#for key, value in df.items():
#    value = sorted(value, reverse=True)[:713]
#    # create a violin plot usin seaborn:
#    import seaborn as sns
#    import matplotlib.pyplot as plt
#    sns.violinplot(data=value)
#    key_title = key.replace('clusts\\cs_25_max_2_hp_10_gc_opt_sleeping_beauty_150_', '').replace('.fasta', '').replace('_', ' ')
#    plt.title(f"{key_title} (Mean: {np.mean(value)})")
#    plt.show()

    
for key, value in df.items():
    # create a violin plot usin seaborn:
    value = sorted(value, reverse=True)[:713]
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots()
    sns.violinplot(data=value, ax=ax)
    print(key)
    try:
        conf_name = key.split('sleeping_beauty_150_')[1] 
        key_title = conf_name.replace("compress_encrypt_high_entropy_dist","high entropy,").replace("bmp_low_entropy_diff_dist", "diff low entropy,").replace("bmp_low_entropy_evo_dist", "evo low entropy,").replace("raptor", "default (raptor),")
    except:
        conf_name = key.replace("clusts/", "")
        key_title = conf_name
    key_title = key_title.replace("sleeping_beauty_", "").replace("seedspacing", "spacing ").replace('_', ' ').replace(" payloadxor", ", payloadxor").replace('.fasta', '').replace("grass", "grass code").replace("cs23", "")
        # Add the mean value to the plot using plt.text()
    plt.text(0.8, 0.9, f"Mean: {np.mean(value):.5f}", transform=ax.transAxes, fontsize=12, ha='center')
    
    #plt.title(f"{key_title}")
    plt.title("Minimum free energy (MFE) of the generated sequences")
    plt.xlabel(key_title, fontsize=12)
    plt.grid(True)
    plt.savefig(f"clusts/mfe_plot/cmp_mfe_violin_{conf_name}.svg", format="svg", dpi=1200)
    plt.savefig(f"clusts/mfe_plot/cmp_mfe_violin_{conf_name}.pdf", bbox_inches="tight")
    plt.show()
    

In [6]:
a = "CCAGTTGCCTCCACGATTCCATTCTATTCATTCGAATAGGGTCAAACCCGTAATCACCGGAGACCCTGTCATGTTTCGTCTCCGTTGTAATCCAGGACAGCCCAAAGGTTCCCGTAGTGAAGCAAGCAAGGCCTCTTAAAGGCGTCCG"
len(a)
import RNA
rna = RNA.fold_compound(a)
pf = rna.pf()[1]
pf